In [1]:
import logging
import warnings
import time
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

warnings.filterwarnings('ignore')

def compute_aic(series, order, name):
    """Fit ARIMA model and return AIC."""
    try:
        model = ARIMA(series, order=order).fit(method_kwargs={'warn_convergence': False})
        aic = model.aic

        # Log results
        logging.info(f"{name} - Order {order}: AIC = {aic:.4f}")
        return {"aic": aic}
    except Exception as e:
        logging.error(f"Error for {name} - Order {order}: {e}")
        return {"aic": None}

def predict_commodity(commodity, csv_path, pdq_low, pdq_high):
    """Process AIC computation for a given commodity."""
    logging.info(f"Processing: {commodity}")
    try:
        df = pd.read_csv(csv_path, index_col='Date', parse_dates=True)
        if 'Low' not in df or 'High' not in df:
            logging.error(f"Missing 'Low' or 'High' columns for {commodity}")
            return {}

        low_results = compute_aic(df['Low'], pdq_low, f"{commodity} Low")
        high_results = compute_aic(df['High'], pdq_high, f"{commodity} High")
        return {"commodity": commodity, "low": low_results, "high": high_results}

    except Exception as e:
        logging.error(f"Error processing {commodity}: {e}")
        return {}

def process_commodities(group, specific_commodities):
    """Process ARIMA modeling for a group of commodities."""
    results = []
    for specific_commodity, pdq_values in specific_commodities.items():
        pdq_low, pdq_high = tuple(pdq_values[1]), tuple(pdq_values[0])
        csv_path = f'../csv/{group}/{specific_commodity}.csv'
        result = predict_commodity(specific_commodity, csv_path, pdq_low, pdq_high)
        if result:
            results.append(result)
    return results

def process_all_groups(groups):
    """Process all commodity groups using parallel execution."""
    start_time = time.time()
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(process_commodities, group, commodities)
            for group, commodities in groups.items()
        ]
        for future in futures:
            try:
                results.extend(future.result())
            except Exception as e:
                logging.error(f"Error processing group: {e}")
    end_time = time.time()
    logging.info(f"Completed all groups in {end_time - start_time:.2f} seconds.")
    return results

# Commodity group data with ARIMA orders
commodity_groups = {
    'rice': {
        'regular_milled_rice': [(25, 1, 0), (22, 0, 19)],
        'well_milled_rice': [(24, 0, 4), (27, 0, 2)],
        'premium_rice': [(6, 1, 10), (7, 1, 27)],
        'special_rice': [(0, 0, 1), (15, 1, 13)]
    },
    'meat': {
        'beef_brisket': [(28, 0, 33), (30, 1, 0)],
        'beef_rump': [(4, 1, 25), (27, 1, 1)],
        'whole_chicken': [(15, 1, 28), (1, 0, 0)],
        'pork_belly': [(45, 0, 37), (35, 0, 0)],
        'pork_kasim': [(39, 0, 0), (33, 0, 29)]
    },
    'fish': {
        'alumahan': [(28, 0, 33), (42, 1, 28)],
        'bangus': [(33, 1, 38), (33, 1, 45)],
        'galunggong': [(28, 0, 0), (28, 1, 0)],
        'tilapia': [(43, 0, 39), (29, 1, 0)]
    },
    'fruits': {
        'banana_lakatan': [(7, 0, 10), (15, 1, 17)],
        'calamansi': [(29, 0, 36), (45, 0, 32)],
        'mango': [(47, 2, 0), (49, 0, 0)],
        'papaya': [(31, 1, 34), (42, 1, 0)]
    },
    'vegetables': {
        'cabbage': [(49, 0, 0), (49, 0, 49)],
        'carrots': [(31, 0, 30), (40, 0, 0)],
        'eggplant': [(31, 1, 39), (35, 0, 0)],
        'tomato': [(0, 0, 15), (40, 0, 0)],
        'white_potato': [(47, 1, 32), (28, 2, 32)]
    },
    'spices': {
        'garlic': [(29, 1, 30), (31, 2, 0)],
        'red_onion': [(1, 1, 27), (5, 1, 11)]
    }
}

# Process all groups and output the results
aic_results = process_all_groups(commodity_groups)
logging.info("All processing completed.")

2024-12-18 19:56:19,839 - INFO - Processing: regular_milled_rice
2024-12-18 19:56:19,842 - INFO - Processing: beef_brisket
2024-12-18 19:56:19,845 - INFO - Processing: alumahan
2024-12-18 19:56:19,849 - INFO - Processing: banana_lakatan
2024-12-18 19:56:19,853 - INFO - Processing: cabbage
2024-12-18 19:56:19,854 - INFO - Processing: garlic
2024-12-18 19:56:48,490 - INFO - beef_brisket Low - Order (30, 1, 0): AIC = 3808.9782
2024-12-18 19:57:00,272 - INFO - garlic Low - Order (31, 2, 0): AIC = 5506.2175
2024-12-18 19:57:54,705 - INFO - banana_lakatan Low - Order (15, 1, 17): AIC = 3198.3810
2024-12-18 19:58:34,870 - INFO - regular_milled_rice Low - Order (22, 0, 19): AIC = 2114.2568
2024-12-18 19:58:41,709 - INFO - banana_lakatan High - Order (7, 0, 10): AIC = 3278.1953
2024-12-18 19:58:41,733 - INFO - Processing: calamansi
2024-12-18 19:58:57,004 - INFO - regular_milled_rice High - Order (25, 1, 0): AIC = 1508.0335
2024-12-18 19:58:57,017 - INFO - Processing: well_milled_rice
2024-12-1

KeyboardInterrupt: 

In [ ]:
aic_results